In [5]:
from ase.io import read, write
import numpy as np

atoms = read("./jmp-s-MgSi-mptrj.xyz", index=0)
print(atoms.get_positions())
print(atoms.get_cell())
print(atoms.info["node_features"])


[[-0.91559356  4.42640257 -0.38316962]
 [ 5.0699048   1.32753575 -2.42969084]
 [-1.11768103  3.44839883  4.15745735]
 [ 0.86092585  2.1631937   1.47511756]
 [ 3.305825    3.30098414 -0.48317614]
 [ 2.38047576  0.44415089 -0.82452267]
 [ 2.61758423  2.89459658 -3.52554584]
 [-4.72391653  5.79787636 -0.09908229]
 [ 4.77054405  1.21628368  0.40756866]
 [ 3.9768486   0.90067512  2.66448069]
 [ 0.02968658  4.90500736  2.166219  ]
 [ 0.82911426  2.60515499 -1.53553665]
 [-2.94854975  5.54240608 -2.08150339]
 [ 2.39447045  4.51521778  1.7342099 ]]
Cell([[6.412071704864502, -0.0038352999836206436, -1.8478816747665405], [-4.85444974899292, 5.803574562072754, -4.606549263000488], [0.0056654298678040504, -0.001001829979941249, 7.560880661010742]])
[[ 2.0362792   0.8575297   0.40736744 ...  2.9537816   1.23237169
   0.9735955 ]
 [ 2.24784517  0.91578138  0.60349762 ...  2.03681254  1.64246643
   1.12261438]
 [ 2.00162029  0.90266544  0.23938972 ...  1.99605453  1.28856623
   1.58742213]
 ...
 [ 2.

In [16]:
from ase.io import read, write
from ase import Atoms
import numpy as np
from ase.calculators.calculator import Calculator
from ase.stress import full_3x3_to_voigt_6_stress

class FakeCalculator(Calculator):
    implemented_properties = ["energy", "free_energy", "forces", "stress"]
    def __init__(self, energy: float, forces: np.ndarray, stress: np.ndarray):
        super().__init__()
        self.energy = energy
        self.forces = forces
        self.stress = stress
    
    def calculate(
        self, atoms=None, properties=None, system_changes=None
    ):
        all_changes = ['positions', 'numbers', 'cell', 'pbc',
               'initial_charges', 'initial_magmoms']

        properties = properties or ["energy"]
        system_changes = system_changes or all_changes
        super().calculate(atoms=atoms, properties=properties, system_changes=system_changes)
        self.results.update({
            "energy": self.energy, 
            "free_energy": self.energy,
            "forces": self.forces, 
            "stress": full_3x3_to_voigt_6_stress(self.stress)
        })

atoms_list = read("./temp_data/water.xyz", index=":")
volume_list = []
num_atoms_list = []
processed_atoms_list = []
for atoms in atoms_list:
    energy = atoms.info["TotEnergy"]
    forces = atoms.arrays["force"]
    stress = np.zeros((3, 3))
    calculator = FakeCalculator(energy, forces, stress)
    processed_atoms = Atoms(symbols=atoms.get_chemical_symbols(), positions=atoms.get_positions(), cell=atoms.get_cell(), pbc=atoms.pbc)
    processed_atoms.set_calculator(calculator)
    energy_ = processed_atoms.get_potential_energy()
    forces_ = processed_atoms.get_forces()
    stress_ = processed_atoms.get_stress(voigt=False)
    assert np.allclose(energy, energy_)
    assert np.allclose(forces, forces_)
    assert np.allclose(stress, stress_)
    processed_atoms_list.append(processed_atoms)
    volume_list.append(processed_atoms.get_volume())
    num_atoms_list.append(len(processed_atoms))
print("Length of atoms_list: ", len(atoms_list))
print("Average volume: {}A^3".format(np.mean(volume_list)))
print("Average num_atoms: ", np.mean(num_atoms_list))
write("./temp_data/water_processed.xyz", processed_atoms_list)

/tmp/ipykernel_4164252/3927738579.py:41: DeprecationWarning: Please use atoms.calc = calc
  processed_atoms.set_calculator(calculator)


Length of atoms_list:  1593
Average volume: 14396.887578476166A^3
Average num_atoms:  192.0


In [3]:
from ase.io import read, write
from ase import Atoms
import numpy as np

atoms_list = read("./temp_data/water_processed.xyz", index=":")
energy_list = [atoms.get_potential_energy() for atoms in atoms_list]
energy_per_atom_list = [energy/len(atoms) for energy, atoms in zip(energy_list, atoms_list)]
e_std = np.std(energy_list)
print("Energy std: ", e_std)
e_per_atom_std = np.std(energy_per_atom_list)
print("Energy per atom std: ", e_per_atom_std)

Energy std:  0.5448013872538504
Energy per atom std:  0.002837507225280458
